In [177]:
# import relevant packages
import geopandas as gpd
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import csv
%matplotlib qt
from itertools import islice
import time

In [ ]:
def distribute(needs,space): #distributes cars, there might be a faster way, but i cannot think of one right now
    endresult=np.zeros(len(needs),dtype=int)
    # filter=needs
    while space>=np.sum((needs>0)):
        add=min(np.min(needs[needs>0]),space//np.sum((needs>0)))
        endresult[needs>0]+=add
        space-=add*np.sum((needs>0))
        needs[needs>0]-=add
    endresult[np.nonzero(needs>0)[0][:space]]+=1
    return endresult
class edge(object):
    def __init__(self,length,speed,lanes,laneboundaries,id=0):
        self.length = int(length)
        self.lanes = int(lanes)
        if laneboundaries==None: laneboundaries=(int(lanes),int(lanes))
        self.basespeed=int(speed)
        self.speed=int(speed)
        self.cars = np.zeros((int(lanes*length/(4.5+speed/1.8)),5)) #collums: occupied,age,remaining travel time, spawnpoint, destination
        self.C0=int(lanes*length/(4.5+speed/1.8))
        self.Cmax=int(laneboundaries[1]*length/(4.5+speed/1.8))
        self.id=id
        self.leftthismin_avg=0
        self.leftthismin=0
        self.minlanes,self.maxlanes=laneboundaries
    def __str__(self): 
        return f"edge {self.id} cars:{np.sum(self.cars[:,0])}"
    def __repr__(self): 
        return f"edge {self.id} cars:{np.sum(self.cars[:,0])}"
    def C(self): #capacity of edge
        return int(self.lanes*self.length/(4.5+self.speed/1.8))
    def tto(self):
        return self.length/self.speed*60/1000
    def update(self):
        Filter = np.nonzero(self.cars[:,0]==1)[0] #Filter for 
        self.cars[Filter,2] -= 1 #travelled one minute
        self.cars[Filter,1] += 1 #aged one minute
        if file:
            if self.size()>0.9*self.C():
                if self.lanes<self.maxlanes:
                    print(f"spitsstrook open op {self}")
                    self.lanes+=1
                    self.cars=np.concatenate((self.cars,np.zeros((self.C()-len(self.cars),5))))
                elif self.size()>0.9*self.Cmax and self.speed>0.5*self.basespeed:
                    print(f"file op {self},speed is now {self.speed}")
                    self.speed-=.1*self.basespeed
                    self.cars=np.concatenate((self.cars,np.zeros((self.C()-len(self.cars),5))))
        self.leftthismin_avg=(1-smoothing)*self.leftthismin_avg+smoothing*self.leftthismin
        self.leftthismin=0
    def tt_avg(self):
        if np.sum(self.cars[:,0])==0: return self.tto()*0.1 #defaults to tt0 if road is empty
        else: return (max(np.sum(self.cars[self.cars[:,2]>0,2]),1)/np.sum(self.cars[:,0])) #otherwise average the traveltime
    def avg_waittime(self):
        """average time a car waits to pass through node"""
        if np.sum(self.cars[:,2]<0)==0: return 0
        else: return -np.average(self.cars[self.cars[:,2]<0,2])
    def tt(self):
        return self.tto()*(1+0.15*(self.size()/self.C())**4)
    def size(self):
        return sum(self.cars[:,0])
class node(object):
    def __init__(self,inputs=[],output=[],destinations=[],id=0,average=85,sd=1.5):
        self.id=id
        self.output = output
        self.input = inputs
        self.destinations=destinations
        self.dest_weights=[]
        self.waitlist=dict(zip(self.output,[dict(zip(self.input,[np.zeros(0,dtype=int) for j in self.input]))for i in self.output]))
        for i in self.waitlist.keys():
            self.waitlist[i]["self"]=np.array([],dtype=int)        
        self.buildup=np.zeros((0,5))
        self.times=np.zeros((0,2)) #array with first collumn as times, and second as origins
        self.choices=np.empty(0,dtype=edge)
        self.dests=np.zeros(0)
        self.gen_average=average
        self.gen_sd=sd
    def initwaitlist(self):
        if len(self.waitlist.keys())>0: print("waitlist already had keys",self.waitlist); return
        else: self.waitlist=dict(zip(self.output,[dict(zip(self.input,[np.zeros(0,dtype=int) for j in self.input]))for i in self.output]))
    def __str__(self): 
        if len(self.buildup): return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]} and buildup:{len(self.buildup)}"
        else: return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]}"
    def __repr__(self): 
        if len(self.buildup): return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]} and buildup:{len(self.buildup)}"
        else: return f"node {self.id} with input: {[i.id for i in self.input]} and output: {[i.id for i in self.output]}"
    def size(self):
        return len(self.buildup)    
    def avg_waittime(self,edge_out):
        if edge_out not in self.output: print(f"what ya doing mate {edge_out} does not exit {self}")
        if age_in_node:carswaiting=np.concatenate([i.cars[self.waitlist[edge_out][i],2] for i in self.input]+[-self.buildup[self.choices==edge_out,1]])
        else:np.concatenate([i.cars[self.waitlist[edge_out][i],2] for i in self.input])
        if len(carswaiting)==0: return 0.00001
        elif take_max:return -np.min(carswaiting)
        else:return -np.average(carswaiting)
    def flow(self):
        for i in self.input: #putting stuff in waitlist
            Remove=np.nonzero(np.logical_and(i.cars[:,4]==self.id,np.logical_and(i.cars[:,2]<=0,i.cars[:,0]==1)))[0]
            self.times=np.concatenate((self.times,i.cars[Remove,:][:,[1,3]])) #remove all cars that have arrived at their destination
            i.cars[Remove,:]=0
            i.leftthismin+=len(Remove)
            # if self.id==2: print(f"recieved {len(Remove)} cars from {i}")
            if self.output==[]:print(f"node {self.id} had no output roads") ;continue
            Filter2 = np.nonzero(np.logical_and(i.cars[:,2]==0,i.cars[:,0]==1))[0] #check which cars arrived now
            if len(self.output)==len(self.input)==2: choices=np.full(len(Filter2),[edges for edges in self.output if edges.id !=(i.id[1],i.id[0])][0]) #cars are not allowed to go back, so this is the only option for cars
            else:
                handling=i.cars[Filter2,:] #cars that need to be handled this function call
                choices=np.empty(len(handling),dtype=edge)
                for dest in np.unique(handling[:,4]):
                    routes=list(islice(nx.shortest_simple_paths(gfd,self.id,dest,weight="tt_avg"),len(self.output)-1))
                    a=[gfd.edges[path[:2]]["object"]for path in routes]
                    weights=1/np.array([nx.path_weight(gfd,path,weight="tt_avg") for path in routes])
                    rander=np.random.choice(a,size=sum(handling[:,4]==dest),p=weights/np.sum(weights)) #randomly choose next edge
                    choices[handling[:,4]==dest]=rander
            for indexes in choices:
                if indexes not in self.output: print("fuck")
            for j in self.output:
                self.waitlist[j][i]=np.concatenate((self.waitlist[j][i],np.array(Filter2[choices==j],dtype=int))) #add new indexes along with their location at the end
        if len(self.buildup)-len(self.choices)>0: #handle buildup caused by generate
            handling=self.buildup[len(self.choices)-len(self.buildup):] #self.buildup[len(self.choices)-len(self.buildup):] should be the cars not handled yet
            if len(self.output)==1: choices=np.full(len(self.buildup)-len(self.choices),self.output[0])
            elif len(self.output)==len(self.input)==2: choices=np.array([gfd.edges[paths[self.id][endgoal][:2]]["object"] for endgoal in handling[:,4]]) #2 edges means we only randomly choose out of 1 option, so might as well take that one immediatly
            else:   
                choices=np.empty(len(handling),dtype=edge)
                for dest in np.unique(handling[:,4]): 
                    routes=list(islice(nx.shortest_simple_paths(gfd,self.id,dest,weight="tt_avg"),len(self.output)-1))
                    a=[gfd.edges[path[:2]]["object"]for path in routes]
                    weights=1/np.array([nx.path_weight(gfd,path,weight="tt_avg") for path in routes])
                    rander=np.random.choice(a,size=sum(handling[:,4]==dest),p=weights/np.sum(weights)) #randomly choose next edge
                    choices[handling[:,4]==dest]=rander
            self.choices=np.concatenate((self.choices,choices))
            self.dests=np.concatenate((self.dests,handling[:,4]))
            if len(self.choices)!=len(self.buildup): print("oi wtf")
            for o in self.output:
                self.waitlist[o]["self"]=np.nonzero(self.choices==o)[0]
                if testing_buildup==True: print(self.waitlist[o]["self"],self,o)
        elif self.id in sources:
            # print("no buildup on",self)
            for o in self.output:
                self.waitlist[o]["self"]=np.array([],dtype=int)
        for i in self.output:#getting stuff out of waitlist
            if sum([len(self.waitlist[i][j]) for j in self.input])+len(self.waitlist[i]["self"])==0: continue #doesn't need to do anything if there are no cars waiting
            elif sum([len(self.waitlist[i][j]) for j in self.input])+len(self.waitlist[i]["self"])<=np.sum(i.cars[:,0]==0):
                for j in self.input:
                    Filter=np.nonzero(i.cars[:,0]==0)[0][:len(self.waitlist[i][j])]
                    j.leftthismin+=len(Filter)
                    i.cars[Filter]=j.cars[self.waitlist[i][j],:] #put cars in next edge
                    i.cars[Filter,2]=np.maximum(np.ceil(i.tt()+np.random.normal(0,i.tto()/10,size=len(Filter))),1)
                    j.cars[self.waitlist[i][j],:]=0 #empty the cars
                    self.waitlist[i][j]=np.zeros(0,dtype=int) #empty waitlist
                Filter=np.nonzero(i.cars[:,0]==0)[0][:len(self.waitlist[i]["self"])]
                i.cars[Filter]=self.buildup[self.waitlist[i]["self"],:]
                i.cars[Filter,2]=np.maximum(np.ceil(i.tt()+np.random.normal(0,i.tto()/10,size=len(Filter))),1)
                self.buildup[self.waitlist[i]["self"],0]=0
            else:
                space=np.sum(i.cars[:,0]==0)
                needs=np.array([len(self.waitlist[i][j]) for j in self.input]+[len(self.waitlist[i]["self"])])
                allowances=distribute(needs,space)
                for k,j in enumerate(self.input):
                    j.leftthismin+=allowances[k]
                    Filter=np.nonzero(i.cars[:,0]==0)[0][:allowances[k]]
                    i.cars[Filter]=j.cars[self.waitlist[i][j][:allowances[k]]] ###probs dont need [Filtertje,:] [Filtertje] seems enough
                    i.cars[Filter,2]=np.maximum(np.ceil(i.tt()+np.random.normal(0,i.tto()/10,size=len(Filter))),1)
                    j.cars[self.waitlist[i][j][:allowances[k]],:]=0 #empty the cars
                    self.waitlist[i][j]=self.waitlist[i][j][allowances[k]:]
                #handle buildup
                Filter=np.nonzero(i.cars[:,0]==0)[0][:allowances[-1]]
                i.cars[Filter]=self.buildup[self.waitlist[i]["self"][:allowances[-1]]] ###probs dont need [Filtertje,:] [Filtertje] seems enough
                i.cars[Filter,2]=np.maximum(np.ceil(i.tt()+np.random.normal(0,i.tto()/10,size=len(Filter))),1)
                self.buildup[self.waitlist[i]["self"][:allowances[-1]],0]=0 #empty the cars
    def generate(self,destinations=[2]): ##change code to account for more generating cities and differing distributions
        N=int(np.random.normal(self.gen_average,self.gen_sd)+0.5) #+.5 rounds it to the nearest integer
        self.choices=self.choices[self.buildup[:,0]!=0]
        self.dests=self.dests[self.buildup[:,0]!=0]
        self.buildup=self.buildup[self.buildup[:,0]!=0]
        if sum(self.buildup[:,4]!=self.dests):print(self,self.buildup,"wtf")
        if age_in_node: self.buildup[:,1]+=1 #only leave if we allow cars to age while waiting in cities
        newcars=np.array([np.ones(N),np.zeros(N),np.random.normal(0,1,size=N),np.full(N,self.id),np.random.choice(self.destinations,N,p=self.dest_weights)]).transpose()
        self.buildup=np.concatenate((self.buildup,newcars)) #needs to handle the built up cars first

In [179]:
testing_buildup=False
# f.write()
# f.closeddsafdadsfsdfsdfsd# 
file=0
age_in_node=True
# tracker={n:0 for n in gfd.nodes}
smoothing=1/10
minutes_init=120
minutes_sim=120

In [180]:
#importing the map
traffic_map = gpd.read_file("./map/2city_shape_v5.shp").to_crs(epsg=4326)
traffic_map['geometry'] = traffic_map['geometry'].scale(xfact= 1/100,yfact =1/100, origin = (0,0))
# traffic_map.iloc[13,4]=2000
#capacity
traffic_map['capciteit'] = traffic_map['lengte']*traffic_map['banen']*1/(4.5+55)

# splitting_tabels
# traffic_map=traffic_map[traffic_map["id"]!=14]
Node_df = pd.concat((traffic_map[traffic_map['Category'] == 'City'].iloc[:,:], traffic_map[traffic_map['Category'] == 'Node'].iloc[:,:])) 
Edge_df = traffic_map[traffic_map['Category'] == 'Edge'].iloc[:,:]
Edge_df["node_start"],Edge_df["node_end"]=0,0
for i in Edge_df["id"]:
    start,end=list(str(np.array(Edge_df['name'].loc[Edge_df['id']==i]))[-4:-2])
    for j in Node_df["name"]:
        if start in j: startid=str(np.array(Node_df['id'].loc[Node_df['name']==j])[0])
        if end in j: endid=str(np.array(Node_df['id'].loc[Node_df['name']==j])[0])
    Edge_df["node_start"][i-1],Edge_df["node_end"][i-1]=int(startid),int(endid)
#printing the tables
# print(traffic_map)
print(Node_df)
print(Edge_df)

   id    name  lengte  maxshd  banen Category  \
0   1   City1     NaN     NaN    NaN     City   
1   2   City2     NaN     NaN    NaN     City   
2   3  Node_A     NaN     NaN    NaN     Node   
3   4  Node_B     NaN     NaN    NaN     Node   
4   5  Node_C     NaN     NaN    NaN     Node   
5   6  Node_D     NaN     NaN    NaN     Node   
6   7  Node_E     NaN     NaN    NaN     Node   

                                            geometry  capciteit  
0  POLYGON ((0.66646 -3.58556, 0.81725 -3.33882, ...        NaN  
1  POLYGON ((13.00588 -2.81356, 13.20662 -2.74107...        NaN  
2  POLYGON ((4.65275 -0.85074, 4.81724 -0.76988, ...        NaN  
3  POLYGON ((4.82561 -6.53287, 5.05702 -6.49384, ...        NaN  
4  POLYGON ((7.45199 -3.78381, 7.6137 -3.68623, 7...        NaN  
5  POLYGON ((10.24566 -0.99293, 10.33488 -0.90092...        NaN  
6  POLYGON ((10.26518 -6.73362, 10.40458 -6.62209...        NaN  
    id     name   lengte  maxshd  banen Category  \
7    8  Edge_1A  30000.0   

C:\Users\kvanu\AppData\Local\Temp\ipykernel_23072\2559309204.py:18: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  Edge_df["node_start"][i-1],Edge_df["node_end"][i-1]=int(startid),int(endid)
C:\Users\kvanu\AppData\Local\Temp\ipykernel_23072\2

In [181]:
#making nodes and edges
gf = nx.Graph()

for _, k in Node_df.iterrows():
    gf.add_node(int(k['id']),
                x=k['geometry'].centroid.x,
                y=k['geometry'].centroid.y,
                stad_naam=k['name'])

for _, row in Edge_df.iterrows():
    gf.add_edge(int(row['node_start']), int(row['node_end']),
                wegnr=row['name'],
                length=row['lengte'],
                maxshd=row['maxshd'],
                nbanen=row['banen'])
gfd = nx.DiGraph(gf)
# nodedict=dict(zip(Node_df['id'],[node([],[],id=i) for i in Node_df['id']]))
# edgedict=dict(zip(Node_df['id'],[dict() for i in Node_df['id']]))

# for i in Edge_df["id"]:

#     start,end=list(str(np.array(Edge_df['name'].loc[Edge_df['id']==i]))[-4:-2])
#     for j in Node_df["name"]:
#         if start in j: startid=str(np.array(Node_df['id'].loc[Node_df['name']==j])[0])
#         if end in j: endid=str(np.array(Node_df['id'].loc[Node_df['name']==j])[0])
#     edgedict[int(startid)][int(endid)]=edge(Edge_df['lengte'][i-1],Edge_df['banen'][i-1],id=i)

# for i in nodedict.keys():
#     nodedict[i].output=[edgedict[i][j]for j in edgedict[i].keys()]
#     for j in edgedict[i].keys():
#         nodedict[j].input.append(edgedict[i][j])
# for i in range(7):
#     nodedict[i+1].initwaitlist()
#     # print(nodedict[i+1].input)

In [182]:
def weight_edge_min(f, t, attr):
    return attr['length']/attr['maxshd']/1000*60

def weight_edge_dist(f, t, attr):
    return attr['length']

In [183]:
sources=[1,2]
Node_df["id"][np.nonzero(Node_df["Category"]=="City")[0]]
# sources=[1000,1001,1002]
nx.set_edge_attributes(gfd,{e: edge(gfd.edges[e]["length"],gfd.edges[e]["maxshd"],gfd.edges[e]["nbanen"],laneboundaries=(gfd.edges[e]["nbanen"],gfd.edges[e]["nbanen"]),id=e) for e in gfd.edges},"object") #put edge objects on the edges
nx.set_node_attributes(gfd,{n: node(inputs=[gfd.edges[e]["object"] for e in gfd.in_edges(n)],output=[gfd.edges[e]["object"] for e in gfd.out_edges(n)],id=n) for n in gfd.nodes},"object") 
#put node objects on nodes
for i in sources:
    gfd.nodes[i]["object"].destinations=list(filter(lambda x:x!=i,sources)) #destination is any city except source
    gfd.nodes[i]["object"].dest_weights=[1]

simulation

In [184]:
nx.set_edge_attributes(gfd,{e: (gfd.edges[e]["object"].tt_avg()+gfd.nodes[e[0]]["object"].avg_waittime(gfd.edges[e]["object"]))*(1+gfd.edges[e]["object"].size()/gfd.edges[e]["object"].C()) for e in gfd.edges},"tt_avg") #put edge objects on the edges

nx.shortest_path(gfd,5,2,weight="tt_avg")
for i in nx.shortest_simple_paths(gfd,5,2):
    print(i,nx.path_weight(gfd,i,weight="tt_avg"))


[5, 2] 1.2000100000000002
[5, 6, 2] 1.8000200000000004
[5, 7, 2] 2.10002


In [185]:
# minutes1=120 #initialisation time
# minutes2=
 #simulation time
tf_df = traffic_map.copy()
# cars=np.zeros((len(tf_df["id"]),minutes2))
sources=[1,2]
# sources=[1000,1001,1002]
try: minutes_sim,minutes_init
except: 
    minutes_init= 120#initialisation time
    minutes_sim=120#simulation time
cars_forw=np.zeros((len(tf_df),minutes_sim))
cars_backw=np.zeros((len(tf_df),minutes_sim))
def tt_avg(f, t, attr):
    return attr['object'].tt_avg()+attr['object'].avg_waittime() #average waittime is remaning time plus wait time at end of edge
for minute in range(minutes_init):
    nx.set_edge_attributes(gfd,{e: gfd.edges[e]["object"].tt_avg()+gfd.nodes[e[0]]["object"].avg_waittime(gfd.edges[e]["object"]) for e in gfd.edges},"tt_avg") #put edge objects on the edges
    paths={n: b for (n, (a, b)) in nx.all_pairs_dijkstra(gfd, weight="tt_avg")}
    for i in sources:
        # print(i,gfd.nodes[i]["object"])
        gfd.nodes[i]["object"].generate('') #generate cars at all sources
        if len(gfd.nodes[i]["object"].buildup)>0:
            if max(gfd.nodes[i]["object"].buildup[:,1])>minute+1:print(f"{i} aged too much on minute {minute}")
    for e in gfd.edges:
        # print(e,gfd.edges[e]["object"])
        gfd.edges[e]["object"].update()
        if len(gfd.edges[e]["object"].cars)>0:
            if max(gfd.edges[e]["object"].cars[:,1])>minute+1: print(f"{e} aged too much on minute {minute}")
    for n in gfd.nodes:
        # print(n,gfd.nodes[n]["object"])
        gfd.nodes[n]["object"].flow()
for i in sources: gfd.nodes[i]["object"].times=np.zeros((0,2))
for minute in range(minutes_sim):
    print(f"{minute=}")
    nx.set_edge_attributes(gfd,{e: (gfd.edges[e]["object"].tt_avg()+gfd.nodes[e[0]]["object"].avg_waittime(gfd.edges[e]["object"])) for e in gfd.edges},"tt_avg") #put edge objects on the edges
    paths={n: b for (n, (a, b)) in nx.all_pairs_dijkstra(gfd, weight="tt_avg")}
    for i in sources:
    # print(i,gfd.nodes[i]["object"])
        gfd.nodes[i]["object"].generate('') #generate cars at all sources
        if len(gfd.nodes[i]["object"].buildup)>0:
            if max(gfd.nodes[i]["object"].buildup[:,1])>minute+1+minutes_init:print(f"{i} aged too much on minute {minute}")
    for e in gfd.edges:
        # print(e,gfd.edges[e]["object"])
        gfd.edges[e]["object"].update()
        if len(gfd.edges[e]["object"].cars)>0:
            if max(gfd.edges[e]["object"].cars[:,1])>minute+1+minutes_init: print(f"{e} aged too much on minute {minute}")
    for n in gfd.nodes:
        # print(n,gfd.nodes[n]["object"])
        gfd.nodes[n]["object"].flow()
    for index,(start,end) in enumerate(zip(Edge_df["node_start"],Edge_df["node_end"])):
        # if (start,end) in gfd.edges:
        cars_forw[index+len(Node_df),minute]=gfd.edges[(start,end)]["object"].size()
        cars_backw[index+len(Node_df),minute]=gfd.edges[(end,start)]["object"].size()
        # else: continue
cars_forw_pd=pd.DataFrame(cars_forw,columns=[f"f{i}" for i in range(minutes_sim)])
cars_backw_pd=pd.DataFrame(cars_backw,columns=[f"b{i}" for i in range(minutes_sim)])
tf_df=pd.concat((tf_df,pd.DataFrame(cars_forw_pd),cars_backw_pd),axis=1)

minute=0
minute=1
minute=2
minute=3
minute=4
minute=5
minute=6
minute=7
minute=8
minute=9
minute=10
minute=11
minute=12
minute=13
minute=14
minute=15
minute=16
minute=17
minute=18
minute=19
minute=20
minute=21
minute=22
minute=23
minute=24
minute=25
minute=26
minute=27
minute=28
minute=29
minute=30
minute=31
minute=32
minute=33
minute=34
minute=35
minute=36
minute=37
minute=38
minute=39
minute=40
minute=41
minute=42
minute=43
minute=44
minute=45
minute=46
minute=47
minute=48
minute=49
minute=50
minute=51
minute=52
minute=53
minute=54
minute=55
minute=56
minute=57
minute=58
minute=59
minute=60
minute=61
minute=62
minute=63
minute=64
minute=65
minute=66
minute=67
minute=68
minute=69
minute=70
minute=71
minute=72
minute=73
minute=74
minute=75
minute=76
minute=77
minute=78
minute=79
minute=80
minute=81
minute=82
minute=83
minute=84
minute=85
minute=86
minute=87
minute=88
minute=89
minute=90
minute=91
minute=92
minute=93
minute=94
minute=95
minute=96
minute=97
minute=98
minute=99
minute=100

In [186]:
for n in gfd.nodes:
    print(len(gfd.nodes[n]["object"].buildup),n)

487 1
170 2
0 3
0 4
0 5
0 6
0 7


In [187]:
nx.set_edge_attributes(gfd,{e: (gfd.edges[e]["object"].tt()+gfd.nodes[e[0]]["object"].avg_waittime(gfd.edges[e]["object"])) for e in gfd.edges},"tt_avg")

In [188]:
for i in gfd.nodes:
    a=gfd.nodes[i]["object"].waitlist
    for k in a.keys():
        for j in a[k].keys():
            if len(a[k][j])>0 :print(k,j,a[k][j],len(a[k][j]))

edge (1, 3) cars:953.0 self [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 23

In [189]:
nx.set_edge_attributes(gfd,{e: (gfd.edges[e]["object"].tt()+gfd.nodes[e[0]]["object"].avg_waittime(gfd.edges[e]["object"])) for e in gfd.edges},"tt")
nx.shortest_path_length(gfd,1,2,"tt")

75.85855993836061

In [190]:
gfd.nodes[1]["object"].avg_waittime(gfd.edges[(1,3)]["object"])

10.0

In [191]:
for e in gfd.nodes:
    print(gfd.nodes[e]["object"].waitlist)

{edge (1, 3) cars:953.0: {edge (3, 1) cars:999.0: array([], dtype=int32), edge (4, 1) cars:1230.0: array([], dtype=int32), 'self': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
     

In [192]:
b=gfd.edges[(3,5)]["object"].cars
sum(edg.cars[:,2]>20)
# b[b[:,2]<0]

377

In [193]:
for e in gfd.edges:
    print(e,gfd.edges[e]["object"].size()/gfd.edges[e]["object"].C())

(1, 3) 0.953953953953954
(1, 4) 0.863063063063063
(2, 6) 0.8348348348348348
(2, 5) 0.933933933933934
(2, 7) 0.14457831325301204
(3, 1) 1.0
(3, 5) 0.9617117117117117
(4, 1) 0.7387387387387387
(4, 5) 0.9754754754754755
(5, 3) 1.0
(5, 4) 0.7382382382382382
(5, 6) 0.6696696696696697
(5, 2) 1.0
(5, 7) 0.5475475475475475
(6, 5) 0.972972972972973
(6, 2) 0.6696696696696697
(7, 5) 0.48348348348348347
(7, 2) 0.7228915662650602


In [194]:
for e in gfd.edges:
    print(gfd.nodes[e[0]]["object"],gfd.edges[e]["object"])
    # gfd.nodes[e[0]]["object"].avg_waittime(gfd.edges[e]["object"])

node 1 with input: [(3, 1), (4, 1)] and output: [(1, 3), (1, 4)] and buildup:487 edge (1, 3) cars:953.0
node 1 with input: [(3, 1), (4, 1)] and output: [(1, 3), (1, 4)] and buildup:487 edge (1, 4) cars:1437.0
node 2 with input: [(5, 2), (6, 2), (7, 2)] and output: [(2, 6), (2, 5), (2, 7)] and buildup:170 edge (2, 6) cars:278.0
node 2 with input: [(5, 2), (6, 2), (7, 2)] and output: [(2, 6), (2, 5), (2, 7)] and buildup:170 edge (2, 5) cars:311.0
node 2 with input: [(5, 2), (6, 2), (7, 2)] and output: [(2, 6), (2, 5), (2, 7)] and buildup:170 edge (2, 7) cars:24.0
node 3 with input: [(1, 3), (5, 3)] and output: [(3, 1), (3, 5)] edge (3, 1) cars:999.0
node 3 with input: [(1, 3), (5, 3)] and output: [(3, 1), (3, 5)] edge (3, 5) cars:1281.0
node 4 with input: [(1, 4), (5, 4)] and output: [(4, 1), (4, 5)] edge (4, 1) cars:1230.0
node 4 with input: [(1, 4), (5, 4)] and output: [(4, 1), (4, 5)] edge (4, 5) cars:1949.0
node 5 with input: [(2, 5), (3, 5), (4, 5), (6, 5), (7, 5)] and output: [(5, 

In [195]:
saving=False

In [196]:
# name2="simplewith_dijkstra+file"
# with open(f"{name2}times","rb") as f:
#     arr=np.load(f,allow_pickle=False)
# print(arr[:,0])
# print(max(arr[:,0]))
# print(min(arr[:,0]))
# print(np.average(arr[:,0]),np.std(arr[:,0]))
# plt.plot()
# plt.hist(arr[:,0],bins=int((max(arr[:,0])-min(arr[:,0]))),color="r")
# # plt.savefig(f"{name2}redraw.jpeg")
# plt.show()

In [197]:
fil="+file" if file else ""

In [198]:
# histogram chec
sourc=1

fil="+file" if file else ""
name="simp_dijkstra"
n_b = gfd.nodes[2]["object"]
print(n_b.times[n_b.times[:,1]==sourc,0])
print(max(n_b.times[n_b.times[:,1]==sourc,0]))
print(min(n_b.times[n_b.times[:,1]==sourc,0]))
print(np.average(n_b.times[n_b.times[:,1]==sourc,0]),np.std(n_b.times[n_b.times[:,1]==sourc,0]))
plt.plot()
plt.hist(n_b.times[n_b.times[:,1]==sourc,0],bins=int((max(n_b.times[n_b.times[:,1]==sourc,0])-min(n_b.times[n_b.times[:,1]==sourc,0]))))
if saving: plt.savefig(f"{name}{fil}{minutes_init}_{minutes_sim}.jpeg")
plt.show()
print()
if saving:
    with open(f"{name}{fil}times{minutes_init}_{minutes_sim}","wb") as f:
        np.save(f,n_b.times,allow_pickle=False)

[ 79.  80.  77. ... 103. 103. 100.]
126.0
59.0
89.23976356797421 11.732950461955541



**Plotting**

In [199]:
#plotting df

# print(tf_df)



Plot_Node_df = pd.concat((tf_df[tf_df['Category'] == 'City'].iloc[:,:], tf_df[tf_df['Category'] == 'Node'].iloc[:,:])) 
Plot_Edge_df = tf_df[tf_df['Category'] == 'Edge'].iloc[:,:]
print(Plot_Edge_df["f100"])

7      949.0
8     1530.0
9     1284.0
10    1809.0
11     465.0
12     236.0
13     333.0
14     574.0
15     103.0
Name: f100, dtype: float64


In [200]:
#adding vehicles
data1= cars_forw.copy()

cars_list = data1[:,0]


traffic_map.insert(3, "cars", cars_list, True)
# traffic_map=traffic_map.drop("cars",axis=1)
#adding textprompts:




def gen_text(string):
    row_index = traffic_map.index[traffic_map['name'] == string]
    if "City" in string:
        return "City " + string[-1]
    elif 'Node' in string:
        if row_index!=None:
            num_of_car = traffic_map.loc[row_index[0], 'cars']
            return '\n' + string[-1] + f'\n cars: {num_of_car}'
        else:
            return '\n' + string[-1] + '\n cars: Not found'
    elif 'Edge' in string:
        if row_index!=None:
            num_of_car = traffic_map.loc[row_index[0], 'cars']
            return f'cars: {num_of_car}'
        else:
            return 'cars: Not found'
    else:
        return ''


traffic_map['textpromt'] = traffic_map['name'].apply(gen_text)




print(traffic_map)

    id     name   lengte    cars  maxshd  banen Category  \
0    1    City1      NaN     0.0     NaN    NaN     City   
1    2    City2      NaN     0.0     NaN    NaN     City   
2    3   Node_A      NaN     0.0     NaN    NaN     Node   
3    4   Node_B      NaN     0.0     NaN    NaN     Node   
4    5   Node_C      NaN     0.0     NaN    NaN     Node   
5    6   Node_D      NaN     0.0     NaN    NaN     Node   
6    7   Node_E      NaN     0.0     NaN    NaN     Node   
7    8  Edge_1A  30000.0   957.0   100.0    2.0     Edge   
8    9  Edge_1B  50000.0  1367.0   100.0    2.0     Edge   
9   10  Edge_AC  40000.0  1254.0   100.0    2.0     Edge   
10  11  Edge_BC  60000.0  1564.0   100.0    2.0     Edge   
11  12  Edge_CD  20000.0   170.0   100.0    2.0     Edge   
12  13  Edge_D2  10000.0   137.0   100.0    2.0     Edge   
13  14  Edge_C2  20000.0   298.0   100.0    1.0     Edge   
14  15  Edge_CE  30000.0   258.0   100.0    2.0     Edge   
15  16  Edge_E2   5000.0    79.0   100.0

In [201]:
fig, ax = plt.subplots(1, figsize=(15, 8),dpi = 200)
cmap = plt.cm.get_cmap('Reds', 120)
cmap2 = plt.cm.get_cmap('Greens', 120)
print(cmap)
#cax = fig.add_axes([0.92, 0.1, 0.02, 0.78])  # Adjust the position and size as needed
data1[1] = 0
cax2 = fig.add_axes([0.06, 0.1, 0.02, 0.78])  # Adjust the position and size as needed
def update(i):
    print(i)
    ax.clear()
    traffic_map["cars"] = data1[:,i]
    traffic_map['textpromt'] = traffic_map['name'].apply(gen_text)
    Plot_Node_df.plot(cmap=cmap2, linewidth=0.4, ax=ax, edgecolor=".4") ###hier moest er ook nog afgerond worden
    #plotting edges colored by fractions of capacity
    Plot_Edge_df.plot(column=Plot_Edge_df[f"f{i}"]/Plot_Edge_df["capciteit"], cmap=cmap, linewidth=0.4, ax=ax, edgecolor=".4",vmin=0,vmax=1)
    bar_info2 = plt.cm.ScalarMappable(cmap=cmap2, norm=plt.Normalize(vmin=0, vmax=np.ceil(Plot_Node_df[f"b{i}"].max()/100)*100))
    cbar = fig.colorbar(bar_info2, cax=cax2)
    
    #creating labels
    traffic_map.apply(lambda x: ax.annotate(text=x['textpromt'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1)
anim = FuncAnimation(fig,update,interval = 100)

# Create a colorbar
cax = fig.add_axes([0.92, 0.1, 0.02, 0.78])  # Adjust the position and size as needed
bar_info = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=100))
bar_info._A = [] ### geen idee wat dit doet
cbar = fig.colorbar(bar_info, cax=cax)

In [ ]:
# exavg = nx.shortest_path_length(gfd, 1, 2, weight='tt_avg')
# exavg
# t = gfd.nodes[2]['object'].times
# t2 = t[t[:,1] == 1][:, 0]
# N = 20
# tavg = np.convolve(t2, np.ones(N)/N, mode='valid')
# # print(tavg)
# plt.plot(t2)
# plt.plot(tavg)
# plt.axhline(y=exavg, ls='--', color='red')
# plt.ylabel("travel time (min)")
# plt.show()

0


0


0


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75


0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
